In [1]:
import os
import csv
import time
import datetime
import random

from collections import Counter
from math import sqrt

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

from bilm import TokenBatcher, BidirectionalLanguageModel, weight_layers, dump_token_embeddings, Batcher

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 256  # 这个值和ELMo的词向量大小一致
    
    hiddenSizes = [128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    optionFile = "modelParams/elmo_options.json"
    weightFile = "modelParams/elmo_weights.hdf5"
    vocabFile = "modelParams/vocab.txt"
    tokenEmbeddingFile = 'modelParams/elmo_token_embeddings.hdf5'
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        self._optionFile = config.optionFile
        self._weightFile = config.weightFile
        self._vocabFile = config.vocabFile
        self._tokenEmbeddingFile = config.tokenEmbeddingFile
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels
    
    def _genVocabFile(self, reviews):
        """
        用我们的训练数据生成一个词汇文件，并加入三个特殊字符
        """
        allWords = [word for review in reviews for word in review]
        wordCount = Counter(allWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        words = [item[0] for item in sortWordCount.items()]
        allTokens = ['<S>', '</S>', '<UNK>'] + words
        with open(self._vocabFile, 'w') as fout:
            fout.write('\n'.join(allTokens))
    
    def _fixedSeq(self, reviews):
        """
        将长度超过200的截断为200的长度
        """
        return [review[:self._sequenceLength] for review in reviews]
    
    def _genElmoEmbedding(self):
        """
        调用ELMO源码中的dump_token_embeddings方法，基于字符的表示生成词的向量表示。并保存成hdf5文件，文件中的"embedding"键对应的value就是
        词汇表文件中各词汇的向量表示，这些词汇的向量表示之后会作为BiLM的初始化输入。
        """
        dump_token_embeddings(
            self._vocabFile, self._optionFile, self._weightFile, self._tokenEmbeddingFile)

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        y = [[item] for item in y]
        trainIndex = int(len(x) * rate)
        
        trainReviews = x[:trainIndex]
        trainLabels = y[:trainIndex]
        
        evalReviews = x[trainIndex:]
        evalLabels = y[trainIndex:]

        return trainReviews, trainLabels, evalReviews, evalLabels
        
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        
#         self._genVocabFile(reviews) # 生成vocabFile
#         self._genElmoEmbedding()  # 生成elmo_token_embedding
        
        reviews = self._fixedSeq(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
                
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(len(data.trainReviews)))
print("train label shape: {}".format(len(data.trainLabels)))
print("eval data shape: {}".format(len(data.evalReviews)))

train data shape: 20000
train label shape: 20000
eval data shape: 5000


In [5]:
# 输出batch数据集
def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
        # 每一个epoch时，都要打乱数据集
        midVal = list(zip(x, y))
        random.shuffle(midVal)
        x, y = zip(*midVal)
        x = list(x)
        y = list(y)

        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX =x[start: end]
            batchY = y[start: end]

            yield batchX, batchY

In [6]:
# 构建模型
class BiLSTMAttention(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.float32, [None, config.sequenceLength, config.model.embeddingSize], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        with tf.name_scope("embedding"):
            embeddingW = tf.get_variable(
                "embeddingW",
                shape=[config.model.embeddingSize, config.model.embeddingSize],
                initializer=tf.contrib.layers.xavier_initializer())
            
            reshapeInputX = tf.reshape(self.inputX, shape=[-1, config.model.embeddingSize])
            
            self.embeddedWords = tf.reshape(tf.matmul(reshapeInputX, embeddingW), shape=[-1, config.sequenceLength, config.model.embeddingSize])
            self.embeddedWords = tf.nn.dropout(self.embeddedWords, self.dropoutKeepProb)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2]
                    self.embeddedWords = tf.concat(outputs_, 2)
        
        outputs = tf.split(self.embeddedWords, 2, -1)
        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self._attention(H)
            outputSize = config.model.hiddenSizes[-1]
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
    
    def _attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes[-1]
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

# 定义计算图

with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        cnn = BiLSTMAttention(config)
        
        # 实例化BiLM对象，这个必须放置在全局下，不能在elmo函数中定义，否则会出现重复生成tensorflow节点。
        with tf.variable_scope("bilm", reuse=True):
            bilm = BidirectionalLanguageModel(
                    config.optionFile,
                    config.weightFile,
                    use_character_inputs=False,
                    embedding_weight_file=config.tokenEmbeddingFile
                    )
        inputData = tf.placeholder('int32', shape=(None, None))
        
        # 调用bilm中的__call__方法生成op对象
        inputEmbeddingsOp = bilm(inputData) 
        
        # 计算ELMo向量表示
        elmoInput = weight_layers('input', inputEmbeddingsOp, l2_coef=0.0)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(cnn.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", cnn.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/textCNN/savedModel")
        sess.run(tf.global_variables_initializer())
    
        def elmo(reviews):
            """
            对每一个输入的batch都动态的生成词向量表示
            """

#           tf.reset_default_graph()
            # TokenBatcher是生成词表示的batch类
            batcher = TokenBatcher(config.vocabFile)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                
                # 生成batch数据
                inputDataIndex = batcher.batch_sentences(reviews)

                # 计算ELMo的向量表示
                elmoInputVec = sess.run(
                    [elmoInput['weighted_op']],
                    feed_dict={inputData: inputDataIndex}
                )

            return elmoInputVec

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            
            feed_dict = {
              cnn.inputX: elmo(batchX)[0],  # inputX直接用动态生成的ELMo向量表示代入
              cnn.inputY: np.array(batchY, dtype="float32"),
              cnn.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              cnn.inputX: elmo(batchX)[0],
              cnn.inputY: np.array(batchY, dtype="float32"),
              cnn.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/textCNN/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(cnn.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(cnn.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(cnn.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

USING SKIP CONNECTIONS
INFO:tensorflow:Summary name embeddingW:0/grad/hist is illegal; using embeddingW_0/grad/hist instead.
INFO:tensorflow:Summary name embeddingW:0/grad/sparsity is illegal; using embeddingW_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell

2019-01-07T19:32:07.026175, step: 47, loss: 0.5496591329574585, acc: 0.6484, auc: 0.8222, precision: 0.8529, recall: 0.4203
2019-01-07T19:32:08.576589, step: 48, loss: 0.5208593606948853, acc: 0.6875, auc: 0.8337, precision: 0.8571, recall: 0.4615
2019-01-07T19:32:10.289107, step: 49, loss: 0.48199906945228577, acc: 0.7578, auc: 0.8461, precision: 0.8421, recall: 0.5614
2019-01-07T19:32:11.984827, step: 50, loss: 0.5084757804870605, acc: 0.7109, auc: 0.8352, precision: 0.85, recall: 0.5231
2019-01-07T19:32:13.665589, step: 51, loss: 0.48681581020355225, acc: 0.7969, auc: 0.862, precision: 0.8372, recall: 0.6545
2019-01-07T19:32:15.303584, step: 52, loss: 0.5239123106002808, acc: 0.6875, auc: 0.8213, precision: 0.6667, recall: 0.5965
2019-01-07T19:32:16.920389, step: 53, loss: 0.5375458598136902, acc: 0.7031, auc: 0.8189, precision: 0.7679, recall: 0.6324
2019-01-07T19:32:18.660985, step: 54, loss: 0.46002197265625, acc: 0.7734, auc: 0.8698, precision: 0.7857, recall: 0.7213
2019-01-07T

2019-01-07T19:35:08.971647, step: 112, loss: 0.41812634468078613, acc: 0.8203, auc: 0.8955, precision: 0.9348, recall: 0.6825
2019-01-07T19:35:10.640816, step: 113, loss: 0.458964079618454, acc: 0.7969, auc: 0.8784, precision: 0.8269, recall: 0.7167
2019-01-07T19:35:12.372822, step: 114, loss: 0.46779465675354004, acc: 0.7656, auc: 0.8616, precision: 0.7692, recall: 0.6897
2019-01-07T19:35:14.035467, step: 115, loss: 0.4172801077365875, acc: 0.7812, auc: 0.8906, precision: 0.8462, recall: 0.6875
2019-01-07T19:35:15.751887, step: 116, loss: 0.42990991473197937, acc: 0.7656, auc: 0.8821, precision: 0.7885, recall: 0.6833
2019-01-07T19:35:17.449226, step: 117, loss: 0.4330533444881439, acc: 0.7891, auc: 0.8799, precision: 0.8333, recall: 0.678
2019-01-07T19:35:19.154455, step: 118, loss: 0.5181000232696533, acc: 0.7344, auc: 0.8367, precision: 0.8627, recall: 0.6197
2019-01-07T19:35:21.483591, step: 119, loss: 0.4686945378780365, acc: 0.7969, auc: 0.8554, precision: 0.871, recall: 0.75
20

2019-01-07T19:37:09.617823, step: 178, loss: 0.4835854172706604, acc: 0.7344, auc: 0.8638, precision: 0.8627, recall: 0.6197
2019-01-07T19:37:11.900036, step: 179, loss: 0.484719842672348, acc: 0.7422, auc: 0.8475, precision: 0.7895, recall: 0.6818
2019-01-07T19:37:14.172660, step: 180, loss: 0.43855419754981995, acc: 0.8438, auc: 0.8976, precision: 0.8333, recall: 0.9028
2019-01-07T19:37:16.182450, step: 181, loss: 0.47969862818717957, acc: 0.7969, auc: 0.8747, precision: 0.7705, recall: 0.7966
2019-01-07T19:37:18.407787, step: 182, loss: 0.4235095977783203, acc: 0.7969, auc: 0.8867, precision: 0.8596, recall: 0.7313
2019-01-07T19:37:20.112804, step: 183, loss: 0.3972729444503784, acc: 0.7969, auc: 0.9076, precision: 0.9348, recall: 0.6515
2019-01-07T19:37:22.152642, step: 184, loss: 0.42737120389938354, acc: 0.7656, auc: 0.8929, precision: 0.8333, recall: 0.6034
2019-01-07T19:37:23.920184, step: 185, loss: 0.36068832874298096, acc: 0.8281, auc: 0.9263, precision: 0.9767, recall: 0.66

2019-01-07T19:40:23.487033, step: 243, loss: 0.4164600670337677, acc: 0.7969, auc: 0.8924, precision: 0.8511, recall: 0.678
2019-01-07T19:40:25.256645, step: 244, loss: 0.41904938220977783, acc: 0.7812, auc: 0.8994, precision: 0.9333, recall: 0.6269
2019-01-07T19:40:27.016296, step: 245, loss: 0.46247798204421997, acc: 0.7812, auc: 0.8671, precision: 0.8679, recall: 0.6866
2019-01-07T19:40:28.744180, step: 246, loss: 0.48344600200653076, acc: 0.7969, auc: 0.8543, precision: 0.8, recall: 0.7742
2019-01-07T19:40:30.457187, step: 247, loss: 0.447444349527359, acc: 0.7656, auc: 0.8644, precision: 0.82, recall: 0.6613
2019-01-07T19:40:32.180101, step: 248, loss: 0.38423416018486023, acc: 0.8438, auc: 0.9145, precision: 0.8644, recall: 0.8095
2019-01-07T19:40:33.942124, step: 249, loss: 0.4129587709903717, acc: 0.8203, auc: 0.8967, precision: 0.8889, recall: 0.7385
2019-01-07T19:40:36.362653, step: 250, loss: 0.38055840134620667, acc: 0.8125, auc: 0.9067, precision: 0.8596, recall: 0.7538
20

...

2019-01-07T19:57:59.780965, step: 699, loss: 0.3479347229003906, acc: 0.8516, auc: 0.9353, precision: 0.8361, recall: 0.85
2019-01-07T19:58:01.540506, step: 700, loss: 0.2810538709163666, acc: 0.8828, auc: 0.9591, precision: 0.8727, recall: 0.8571

Evaluation:
2019-01-07T19:59:10.738283, step: 700, loss: 0.34041016835432786, acc: 0.8459589743589745, auc: 0.9336410256410256, precision: 0.9001743589743588, recall: 0.7810128205128205
2019-01-07T19:59:12.463846, step: 701, loss: 0.33732765913009644, acc: 0.8281, auc: 0.9303, precision: 0.8545, recall: 0.7705
2019-01-07T19:59:14.253041, step: 702, loss: 0.2996540069580078, acc: 0.9219, auc: 0.9427, precision: 0.96, recall: 0.8571
2019-01-07T19:59:16.114288, step: 703, loss: 0.4371894299983978, acc: 0.7734, auc: 0.8893, precision: 0.9024, recall: 0.5968
2019-01-07T19:59:17.882704, step: 704, loss: 0.3464062213897705, acc: 0.7656, auc: 0.9451, precision: 0.9167, recall: 0.6286
2019-01-07T19:59:19.575442, step: 705, loss: 0.3424730896949768, a

2019-01-07T20:01:04.656148, step: 764, loss: 0.25104930996894836, acc: 0.8984, auc: 0.9607, precision: 0.9815, recall: 0.8154
2019-01-07T20:01:06.454144, step: 765, loss: 0.4526858627796173, acc: 0.7969, auc: 0.8848, precision: 0.8958, recall: 0.6719
2019-01-07T20:01:08.289542, step: 766, loss: 0.2533494234085083, acc: 0.9062, auc: 0.9619, precision: 0.931, recall: 0.871
2019-01-07T20:01:10.059985, step: 767, loss: 0.293709397315979, acc: 0.8672, auc: 0.9464, precision: 0.9074, recall: 0.8033
2019-01-07T20:01:11.860685, step: 768, loss: 0.33804434537887573, acc: 0.8203, auc: 0.936, precision: 0.9038, recall: 0.7231
2019-01-07T20:01:13.636679, step: 769, loss: 0.2573462724685669, acc: 0.8984, auc: 0.9624, precision: 0.9298, recall: 0.8548
2019-01-07T20:01:15.469838, step: 770, loss: 0.23690593242645264, acc: 0.9297, auc: 0.9673, precision: 0.9565, recall: 0.9167
2019-01-07T20:01:17.300234, step: 771, loss: 0.31719231605529785, acc: 0.8984, auc: 0.9533, precision: 0.8594, recall: 0.9322


2019-01-07T20:04:14.866533, step: 828, loss: 0.2554095685482025, acc: 0.8984, auc: 0.9618, precision: 0.9388, recall: 0.8214
2019-01-07T20:04:16.715288, step: 829, loss: 0.4225353002548218, acc: 0.8281, auc: 0.9094, precision: 0.7925, recall: 0.7925
2019-01-07T20:04:18.520263, step: 830, loss: 0.24015021324157715, acc: 0.9141, auc: 0.9652, precision: 0.9455, recall: 0.8667
2019-01-07T20:04:20.428878, step: 831, loss: 0.31889042258262634, acc: 0.8203, auc: 0.9427, precision: 0.8974, recall: 0.6481
2019-01-07T20:04:22.300191, step: 832, loss: 0.29529890418052673, acc: 0.8516, auc: 0.9519, precision: 0.9123, recall: 0.7879
2019-01-07T20:04:24.189859, step: 833, loss: 0.33030831813812256, acc: 0.8516, auc: 0.9404, precision: 0.9388, recall: 0.7419
2019-01-07T20:04:25.992406, step: 834, loss: 0.3482765853404999, acc: 0.8516, auc: 0.929, precision: 0.931, recall: 0.7826
2019-01-07T20:04:27.825663, step: 835, loss: 0.31254878640174866, acc: 0.8906, auc: 0.9576, precision: 0.8281, recall: 0.94

2019-01-07T20:06:19.252789, step: 894, loss: 0.34591928124427795, acc: 0.8906, auc: 0.9379, precision: 0.8395, recall: 0.9855
2019-01-07T20:06:21.162809, step: 895, loss: 0.279893696308136, acc: 0.8828, auc: 0.9631, precision: 0.8667, recall: 0.9286
2019-01-07T20:06:23.010579, step: 896, loss: 0.2198055535554886, acc: 0.9062, auc: 0.9749, precision: 0.9394, recall: 0.8857
2019-01-07T20:06:24.872689, step: 897, loss: 0.28938210010528564, acc: 0.8672, auc: 0.9492, precision: 0.9016, recall: 0.8333
2019-01-07T20:06:26.693091, step: 898, loss: 0.30613917112350464, acc: 0.8516, auc: 0.9461, precision: 0.8824, recall: 0.7759
2019-01-07T20:06:28.564584, step: 899, loss: 0.33581751585006714, acc: 0.8672, auc: 0.9309, precision: 0.918, recall: 0.8235
2019-01-07T20:06:30.431841, step: 900, loss: 0.2455681562423706, acc: 0.8906, auc: 0.9645, precision: 0.96, recall: 0.8

Evaluation:
2019-01-07T20:07:44.416160, step: 900, loss: 0.4159230880248241, acc: 0.8227153846153845, auc: 0.9323358974358974, 

2019-01-07T20:09:36.668534, step: 959, loss: 0.29198628664016724, acc: 0.8828, auc: 0.9442, precision: 0.9194, recall: 0.8507
2019-01-07T20:09:38.576328, step: 960, loss: 0.3308532238006592, acc: 0.875, auc: 0.9374, precision: 0.8676, recall: 0.8939
2019-01-07T20:09:40.461667, step: 961, loss: 0.3285728693008423, acc: 0.8516, auc: 0.934, precision: 0.9062, recall: 0.8169
2019-01-07T20:09:42.372627, step: 962, loss: 0.22361217439174652, acc: 0.8828, auc: 0.9706, precision: 0.9333, recall: 0.8358
2019-01-07T20:09:44.260062, step: 963, loss: 0.309948205947876, acc: 0.8984, auc: 0.9443, precision: 0.8889, recall: 0.9032
2019-01-07T20:09:46.200639, step: 964, loss: 0.32875654101371765, acc: 0.8672, auc: 0.9362, precision: 0.8814, recall: 0.8387
2019-01-07T20:09:48.148770, step: 965, loss: 0.30561962723731995, acc: 0.8906, auc: 0.9433, precision: 0.9322, recall: 0.8462
2019-01-07T20:09:50.138989, step: 966, loss: 0.3292686939239502, acc: 0.8438, auc: 0.935, precision: 0.9412, recall: 0.7385
